<a href="https://colab.research.google.com/github/meliksahb/Design-of-Intelligent-Machines-ME536-/blob/main/ME536_Week4_SubspaceAngles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Please __read the document carefuly__ and submit your code accordingly.  

Once you fill in the functions as explained below, disconnect runtime, reconnect and run all (i.e. ```Runtime -> Run all```) in order to double check that it still works. Delete any test code you have so that I do just see the functions below, I will write my own test code, clear all outuput (i.e. ```Edit -> Clear all outputs```), finally save this file as "**W4_studentID.ipynb**" and submit via [ODTU class](https://odtuclass2024f.metu.edu.tr/mod/assign/view.php?id=68205).  

Use of AI tools (such as the built-in Gemini in colab, or anyother you like) is allowed. However, if you use an AI tool, add the prompt(s) you used as a comment to the beginning of each code cell.  
Along with the comment, explain if the first prompt worked, if not explain how you fixed it, add all versions of your prompts in to your comments.


Allowed imports:  
- ```numpy```.  

Any submission:  
- with test code,  
- that crashes,  
- any other import than mentioned above
- not properly named

will not be graded.


## Full name: Melikşah Beşir

## Student ID: e2738425

## Definition:
You are to complete following functions. Details are explained in the docstring of each funciton.  

In the text cell before each function, in brief but sufficient detail, explain how the function calculates the desired outputs.

Note that these functions complement each other, you can test one with the other if they are properly written.  

Also note that, there is no limit for the dimension of ambient space.  

For the time being, there is no noise in data, i.e. rank from a particular subspace directly indicate the dimension of that subspace.  


### Generation of subspaces with desired guaranteed minimal or maximal angles

Replace the content of this cell where you briefly but sufficiently explain how you make sure that your code works using mathematical terms as much as possible.  

Note that you can write math type expression between $ $.  Recall numpy tutorial on ODTU Class.  

Anyway here are some examples just in case:  
$e = Mc^2$   

$\int sin(t) dt = -cos(t)$   

$\begin{bmatrix} 1 & 3 \\ 2 & 4 \end{bmatrix} \mathbf{x} = \begin{bmatrix} 4 \\ 6 \end{bmatrix}$

$\tilde{x} = 0.999 x$

In [ ]:
import numpy as np
def GenerateSubspacesMin(D = 5, dS1 = 2, dS2 = 2, nS1 = 50, nS2 = 50, minimalAngle = 0 ):
    '''
    This function generates 2 data matrices in subspaces S1 and S2 recpectively that live in D dimensional ambient space
    dSi is the dimension of subspace Si
    nSi is the number of data points to be generated in subspace Si
    minimalAngle is the minimal angle between subspaces, recall the definition in lecture notes
    Function returns two numpy arrays: M1, M2
    where:
        dimension of matrix Mi is (D, nSi)
        rank(Mi) = dSi

    if passed data does not make sense, return two empty numpy arrays
    '''
    # I copy-paste docstring and write "can you explain step by step what I should do for this task"
    # to chatgpt. Then I tried to write code with the help of documentation and stack overflow.
    # I also used ChatGPT for generating explainings of my code. I wrote ChatGPT
    # "Can you prepare me README file with mathematical expression for this code [copy paste code]"
    # Validate inputs
    if dS1 > D or dS2 > D or minimalAngle < 0 or minimalAngle > 90:
        return np.array([]), np.array([])

    # Generate random matrices for each subspace
    random_matrix_1 = np.random.randn(D, dS1)
    random_matrix_2 = np.random.randn(D, dS2)

    # Use QR decomposition to ensure the matrices are orthogonal
    # I find it in stack overflow https://stackoverflow.com/questions/74401008/qr-decomposition
    # Also, https://numpy.org/doc/2.1/reference/routines.linalg.html
    Q1, _ = np.linalg.qr(random_matrix_1)
    Q2, _ = np.linalg.qr(random_matrix_2)

    # Generate data points in subspace S1 and S2
    M1 = Q1[:, :dS1] @ np.random.randn(dS1, nS1)
    M2 = Q2[:, :dS2] @ np.random.randn(dS2, nS2)

    # Apply rotation if minimalAngle > 0 to ensure the angle between the subspaces
    if minimalAngle > 0:
        # Calculate the rotation angle in radians
        angle_rad = np.deg2rad(minimalAngle)
        rotation_matrix = np.eye(D)

        # Apply rotation only in a D=2 subspace to maintain minimal angle
        rotation_matrix[:2, :2] = np.array([[np.cos(angle_rad), -np.sin(angle_rad)],
                                            [np.sin(angle_rad), np.cos(angle_rad)]])
        # Rotate Q2 by this angle
        Q2_rotated = rotation_matrix @ Q2
        M2 = Q2_rotated[:, :dS2] @ np.random.randn(dS2, nS2)
    return M1, M2

This function generates two data matrices, M1 and M2, each representing data points in subspaces S1 and S2 of a D-dimensional ambient space. The generated subspaces have a user-specified minimal angle between them.

Random matrices R1 and R2 are generated of sizes (D,dS1) and (D,dS2) to provide an initial basis for the subspaces.

QR decomposition is applied to each random matrix:

R1=Q1⋅R1reduced,

R2=Q2⋅R2reduced

where Q1 and Q2 are orthonormal matrices with columns that form bases for subspaces S1 and S2.

If a minimal angle is specified, a rotation matrix is created and applied to ensure that the angle between the first basis vectors in Q1 and Q2 matches the desired minimal angle.


### Finding minimal and maximal angles between subspaces  
Similar to the case above, replace the content of this cell to explain briefly but sufficiently how you make sure that your code works using mathematical terms as much as possible.  


In [ ]:

def FindMinimalAngles(M1, M2):
    '''
    This function calculates and returns the minimal angle between subspaces S1, and S2
    that contain the data points M1 and M2 respcetively.
    Check out the lecture notes for the definition of minimal angle
    Note that, subspaces are not be passed, indeed, 2 data matrices are given to this function
    If you have properly have written the previous function, you will be able to
    use it to test this function.
    if passed data does not make sense, return -1000
    '''
    # I copy-paste docstring and write "can you explain step by step what I should do for this task"
    # to chatgpt. Then I tried to write code with the help of documentation and stack overflow
    # Validate inputs
    if M1.shape[0] != M2.shape[0] or M1.size == 0 or M2.size == 0:
        return -1000

    # Perform QR decomposition on M1 and M2 to obtain orthonormal bases
    # I find it in stack overflow https://stackoverflow.com/questions/74401008/qr-decomposition
    Q1, _ = np.linalg.qr(M1)
    Q2, _ = np.linalg.qr(M2)

    # Initialize minimum angle to a large value
    minimal_angle = 90

    # Calculate angles between each pair of basis vectors in Q1 and Q2
    for v1 in Q1.T:  # Each column in Q1
        for v2 in Q2.T:  # Each column in Q2
            # Calculate the cosine of the angle between v1 and v2
            cos_theta = np.clip(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)), -1, 1)
            # Calculate the angle in degrees
            theta = np.degrees(np.arccos(cos_theta))
            # Update minimal angle if a smaller angle is found
            minimal_angle = min(minimal_angle, theta)
    return minimalAngle


This function calculates the minimal angle between the subspaces spanned by the data points in matrices M1 and M2. The function uses QR decomposition to obtain orthonormal bases and iteratively calculates the smallest angle between the bases without using SVD.

QR decomposition is used to obtain orthonormal basis matrices Q1 and Q2 for the column spaces of M1 and M2:

M1=Q1⋅M1reduced

M2=Q2⋅M2reduced

The angle θ between any two basis vectors v1 from Q1 and v2 from Q2 is calculated using the cosine of the angle:

cos(θ)=(v1⋅v2)/(∥v1∥⋅∥v2∥)

This cosine value is converted to an angle in degrees. The function returns the smallest angle found between any pair of basis vectors from Q1 and Q2.
